In [ ]:
import os
import numpy as np
import random
import pandas as pd
import pygad
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam, RMSprop
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Dados
df = pd.read_csv('data_per_month.csv')['Total Renewable Energy'].values
window_size = 12

def create_windows(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i + window_size])
        y.append(data[i + window_size])
    return np.array(X), np.array(y)

def remove_outliers(X, y, threshold=3):
    mask_X = np.all(np.abs(X - np.mean(X, axis=0)) <= threshold * np.std(X, axis=0), axis=1)
    mask_y = np.abs(y - np.mean(y)) <= threshold * np.std(y)
    mask = mask_X & mask_y
    return X[mask], y[mask]

X, y = create_windows(df, window_size)
X, y = remove_outliers(X, y)

train_size = int(len(X) * 0.7)
val_size = int(len(X) * 0.15)
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test = X[train_size+val_size:], y[train_size+val_size:]

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Criar modelo
def build_model(neurons, dense_layers, optimizer_code):
    model = Sequential()
    model.add(Input(shape=(X_train_scaled.shape[1],)))
    for _ in range(dense_layers):
        model.add(Dense(neurons, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1))
    optimizer = Adam() if optimizer_code == 0 else RMSprop()
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return model

# Função de avaliação
def fitness_func(ga_instance, solution, sol_idx):
    try:
        neurons = int(solution[0])
        dense_layers = int(solution[1])
        optimizer_code = int(solution[2])
        batch_size = int(solution[3])
        epochs = int(solution[4])

        print(f"[FITNESS] Neurons={neurons}, Layers={dense_layers}, Opt={'Adam' if optimizer_code==0 else 'RMSprop'}, Batch={batch_size}, Epochs={epochs}")

        model = build_model(neurons, dense_layers, optimizer_code)
        model.fit(X_train_scaled, y_train, validation_data=(X_val_scaled, y_val),
                  epochs=epochs, batch_size=batch_size, verbose=0)

        preds = model.predict(X_val_scaled, verbose=0)
        mse = mean_squared_error(y_val, preds)
        return 1.0 / (mse + 1e-8)
    except Exception as e:
        print(f"[ERRO NA FITNESS] {e}")
        return 0

# Espaço de busca
gene_space = [
    {'low': 8, 'high': 128},     # neurônios
    {'low': 1, 'high': 4},       # número de camadas densas
    {'low': 0, 'high': 1},       # otimizador (0=Adam, 1=RMSprop)
    {'low': 8, 'high': 64},      # batch size
    {'low': 1000, 'high': 4000}     # épocas
]

# Instância do GA
ga_instance = pygad.GA(
    num_generations=10,
    num_parents_mating=4,
    fitness_func=fitness_func,
    sol_per_pop=10,
    num_genes=len(gene_space),
    gene_space=gene_space,
    parent_selection_type="rank",
    crossover_type="single_point",
    mutation_type="random",
    mutation_percent_genes=20,
    on_generation=lambda ga: print(f"Ger {ga.generations_completed} - Melhor fitness: {ga.best_solution()[1]:.6f}")
)

ga_instance.run()

# Melhor solução
solution, fitness, _ = ga_instance.best_solution()
print("\nMelhor conjunto de hiperparâmetros:")
print(f"Neurônios: {int(solution[0])}")
print(f"Camadas densas: {int(solution[1])}")
print(f"Otimizador: {'Adam' if int(solution[2]) == 0 else 'RMSprop'}")
print(f"Batch size: {int(solution[3])}")
print(f"Épocas: {int(solution[4])}")
print(f"Fitness: {fitness}")


[FITNESS] Neurons=33, Layers=2, Opt=Adam, Batch=39, Epochs=2418
